# Giới thiệu về dự án
Dự án tạo ra cơ sở dữ liệu về dược liệu phục vụ cho quá trình giảng dậy và nghiên cứu. Tầm nhìn dự án tạo ra hệ thống phân tích sâu hơn về dược dân tộc học hướng tới tìm kiếm các hoạt chất tiềm năng làm thuốc. Các bước chuẩn bị gồm có:
- Chuẩn hóa dữ liệu
- Phân tích dữ liệu
Chuẩn hóa dữ liệu sẽ được update tới website để nhân phản hồi của cộng đồng. 

## Chuẩn hóa dữ liệu

Các hàm cấn thiết dành cho hầu hết các chương trình chạy

In [1]:
# Hàm để chuyển đổi dòng DataFrame thành nội dung Markdown
def convert_row_to_markdown(row, columns):
    markdown_content = ""
    for col in columns:
        markdown_content += f"### {col}\n{row[col]}\n\n"
    return markdown_content
def get_first_words(text):
    parts = text.split(' ')
    if len(parts) >= 2:
        return ' '.join(parts[:2])
    return text  # Trả về chuỗi gốc nếu có ít hơn 2 từ
def sanitize_filename(name):
    # Thay thế các ký tự không hợp lệ bằng dấu gạch dưới
    return "".join(c if c.isalnum() or c in " ._-" else "_" for c in name)

### câu hình folder 

In [2]:
import os
root = "docs"
folder_markdocs="Markdocs"
Database= "Database"
folder_GIBF = "GIBF_database"
GIBF_database_dir = os.path.join(root, Database, folder_GIBF)
if not os.path.exists(GIBF_database_dir):
    os.makedirs(GIBF_database_dir)
folder_lotus = "Lotus_database"
lotus_database_dir = os.path.join(root, Database, folder_lotus)
if not os.path.exists(lotus_database_dir):
    os.makedirs(lotus_database_dir)
folder_WIKI = "WIKI_database"
WIKI_database_dir = os.path.join(root, Database, folder_WIKI)
if not os.path.exists(WIKI_database_dir):
    os.makedirs(WIKI_database_dir)
project_DL= "Dược liệu"
markdocs_DL_dir = os.path.join(root, folder_markdocs, project_DL)
if not os.path.exists(markdocs_DL_dir):
    os.makedirs(markdocs_DL_dir)

project_DDT = "Dược dân tộc"
markdocs_DTH_dir = os.path.join(root, folder_markdocs, project_DDT)
if not os.path.exists(markdocs_DTH_dir):
    os.makedirs(markdocs_DTH_dir)

### Các nguồn dữ liệu đầu vào sẵn có
Đầu tiền là bộ cơ sở dữ liệu tự xây dựng được lưu dưới dạng sqlite. 
Bộ dữ liệu của dr Duke về dược dân tộc học.

In [3]:
# Bộ dữ liệu tự xây dựng từ SQLITE
# Bộ dữ liệu do các bạn sinh viên phối hợp thực hiện
import sqlite3
import pandas as pd

# Kết nối đến cơ sở dữ liệu SQLite
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()
# danh sách các bảng trong sqlit3
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") 
tables = cursor.fetchall()
# Truy vấn dữ liệu từ bảng
cursor.execute("SELECT * FROM CSDLHM_loaithucvat")
rows = cursor.fetchall()

# Lấy tên các cột
column_names = [description[0] for description in cursor.description]
# Chuyển đổi dữ liệu thành DataFrame
df_CDSL_Loai = pd.DataFrame(rows, columns=column_names).dropna(subset=['Tên_Loài_Tiếng_Việt'])
# Truy vấn dữ liệu từ bảng
cursor.execute("SELECT * FROM CSDLHM_herbalmedicine")
rows = cursor.fetchall()
# Lấy tên các cột
column_names = [description[0] for description in cursor.description]
# Chuyển đổi dữ liệu thành DataFrame
df_CSDLHM_herbalmedicine = pd.DataFrame(rows, columns=column_names).dropna(subset=['Dược_liệu_Tiếng_Việt'])
df_CSDLHM_herbalmedicine = df_CSDLHM_herbalmedicine.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_CSDLHM_herbalmedicine['Dược_liệu_Tiếng_Latin_Kiểu_DĐVN'] = df_CSDLHM_herbalmedicine['Dược_liệu_Tiếng_Latin_Kiểu_DĐVN'].str.lower().str.replace(r'\s{2,}', ' ', regex=True).str.strip()
df_CSDL_SV = pd.read_excel('Duoc lieu.xlsx')
df_CSDL_SV = df_CSDL_SV.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_CSDL_SV['Tên khoa học của Dược liệu'] = df_CSDL_SV['Tên khoa học của Dược liệu'].str.lower().str.replace(r'\s{2,}', ' ', regex=True).str.strip()
df_CSDL_SV['Tên tiếng việt dược liệu'] = df_CSDL_SV['Tên tiếng việt dược liệu'].str.lower().str.replace(r'\s{2,}', ' ', regex=True).str.strip().str.title()
name0=df_CSDLHM_herbalmedicine.columns[6]
name0
name1= df_CSDL_SV.columns[3]
df_CSDL_SV.rename(columns={name1: name0}, inplace=True)
# Sử dụng merge với how='outer'
df_CSDLHM = pd.merge(df_CSDLHM_herbalmedicine, df_CSDL_SV, on=[name0], how='right')

df_CSDLHM = df_CSDLHM.dropna(subset=['Tên khoa học của Thực vật trong dược điển Việt Nam V'])
df_CSDLHM['Tên khoa học của Thực vật trong dược điển Việt Nam V'] = df_CSDLHM['Tên khoa học của Thực vật trong dược điển Việt Nam V'].apply(get_first_words)
df_CSDLHM['Loài thay thế 1'] = df_CSDLHM['Loài thay thế 1'].fillna('').apply(get_first_words)
df_CSDLHM['Loài thay thế 2'] = df_CSDLHM['Loài thay thế 2'].fillna('').apply(get_first_words)
df_CSDLHM['Loài thay thế 3'] = df_CSDLHM['Loài thay thế 3'].fillna('').apply(get_first_words)
def filter_alternative_species(row): 
    return [value for value in [row['Tên khoa học của Thực vật trong dược điển Việt Nam V'], row['Loài thay thế 1'], row['Loài thay thế 2'], row['Loài thay thế 3']] if pd.notnull(value) and str(value).strip()] 
df_CSDLHM.loc[:,'list_species'] = df_CSDLHM.apply(filter_alternative_species, axis=1)


In [4]:
len(df_CSDLHM)

273

Dịch thuật activatiy tiếng anh sang tiếng Việt. Không cần chạy lại

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
from requests.exceptions import ConnectTimeout
import os

file_path_activity = "E:\\Database\\0-DataNaturalProduct\\dr-dukes-phytochemical-and-ethnobotanical-databases-All-2023-02-13_0033\\Duke-Source-CSV\\ACTIVITIES.csv"
try:
    df_Activity = pd.read_csv(file_path_activity, encoding='utf-8')
except ValueError:
    df_Activity = pd.read_csv(file_path_activity, encoding='latin1')

# Tạo một đối tượng GoogleTranslator
translator = GoogleTranslator(source='auto', target='vi')

def safe_translate(text, dest='vi'):
    for _ in range(3):  # Thử lại tối đa 3 lần
        try:
            translation = translator.translate(text)
            return translation
        except ConnectTimeout:
            print("Kết nối bị timeout, thử lại...")
    return "N/A"  # Trả về "N/A" nếu tất cả các lần thử đều thất bại

# Dịch cột 'ACTIVITY' sang tiếng Việt và lưu vào cột mới 'ACTIVITY_VI'
df_Activity['ACTIVITY_VI'] = df_Activity['ACTIVITY'].apply(lambda x: safe_translate(x))

# Lưu DataFrame đã dịch vào tệp CSV
root = "docs"
Database = "Database"
GIBF_database_dir = os.path.join(root, Database)

df_Activity_save_csv = os.path.join(GIBF_database_dir, "Activity_translate_vi.csv")
if not os.path.exists(GIBF_database_dir):
    os.makedirs(GIBF_database_dir)
df_Activity.to_csv(df_Activity_save_csv, index=False, encoding='utf-8-sig')

print(f"DataFrame đã được lưu vào tệp {df_Activity_save_csv}")


In [5]:
# bộ dữ liệu của dr DUKE
import pandas as pd

# Đường dẫn tới tệp CSV
file_path_entha = "E:\\Database\\0-DataNaturalProduct\\dr-dukes-phytochemical-and-ethnobotanical-databases-All-2023-02-13_0033\\Duke-Source-CSV\\ETHNOBOT.csv"
df_Activity_save_csv = os.path.join(root, Database,"Activity_translate_vi.csv")
try:
    df_Entha = pd.read_csv(file_path_entha, encoding='utf-8')
except ValueError:
    df_Entha = pd.read_csv(file_path_entha, encoding='latin1')
try:
    df_Activity = pd.read_csv(df_Activity_save_csv, encoding='utf-8')
except ValueError:
    df_Activity = pd.read_csv(df_Activity_save_csv, encoding='latin1')



df_Entha = pd.merge(df_Entha, df_Activity, on="ACTIVITY", how = "inner", suffixes=('_df1', '_df2'))
# Nhóm dữ liệu theo các cột 'GENUS', 'SPECIES', 'FAMILY', 'COUNTRY' và tính gộp (aggregate) các cột khác
df_Entha_group = df_Entha.groupby(['GENUS', 'SPECIES', 'FAMILY', 'COUNTRY']).agg({
    'ACTIVITY': list,
    'DEFINITION': list, 
    'ACTIVITY_VI': list
}).reset_index()
# Biến đổi các giá trị trong cột 'ACTIVITY' và 'DEFINITION_df1' từ dạng danh sách sang dạng chuỗi
df_Entha_group['ACTIVITY'] = df_Entha_group['ACTIVITY'].apply(lambda x: ', '.join(map(str, x)))
df_Entha_group['ACTIVITY_VI'] = df_Entha_group['ACTIVITY_VI'].apply(lambda x: ', '.join(map(str, x)))
df_Entha_group['DEFINITION'] = df_Entha_group['DEFINITION'].apply(lambda x: ', '.join(map(str, x)))
df_Entha_group['Country_Enth']= df_Entha_group['COUNTRY']  + ": "+ df_Entha_group['ACTIVITY_VI']
df_Entha_group = df_Entha_group.groupby(['GENUS', 'SPECIES', 'FAMILY']).agg({
    'Country_Enth': list,
    'COUNTRY': set,
    'DEFINITION': set,
    'ACTIVITY_VI': set
}).reset_index()
df_Entha_group['Sciences_Name']= df_Entha_group["GENUS"] + ' ' + df_Entha_group['SPECIES']
# Một số lệnh đảm bảo dữ liệu đạt yêu cầu để tải và lưu file 
import os
import pandas as pd
from translate import Translator
from tabulate import tabulate
def remove_invalid_chars(input_string):
    invalid_chars = "!@#$%^&*()_+-=[]{}|;:'\",.<>?/\\`~"
    translation_table = str.maketrans('', '', invalid_chars)
    return input_string.translate(translation_table)
df_Entha_group['Sciences_Name'] = df_Entha_group['Sciences_Name'].apply(remove_invalid_chars)

In [6]:
df_Entha_group.head(10)

,GENUS,SPECIES,FAMILY,Country_Enth,COUNTRY,DEFINITION,ACTIVITY_VI,Sciences_Name
0,Aaragus,adscendens,Liliaceae,"[Elsewhere: Chất làm lạnh, Làm dịu, India: Thu...","{Iran, Elsewhere, India}","{Diaphoretic, Stimulant, Aphrodisiac, Tonic, R...","{Đổ mồ hôi, Chất kích thích, Chất làm lạnh, Là...",Aaragus adscendens
1,Aaragus,filicinus,Liliaceae,"[Elsewhere: Chất làm se, Thuốc bổ]",{Elsewhere},"{Astringent, Tonic}","{Chất làm se, Thuốc bổ}",Aaragus filicinus
2,Aaragus,lucidum,Liliaceae,"[China: Thuốc an thần, Thuốc bổ]",{China},"{Sedative, Tonic}","{Thuốc an thần, Thuốc bổ}",Aaragus lucidum
3,Aaragus,lucidus,Liliaceae,"[China: Thuốc lợi tiểu, Thuốc long đờm, Thuốc ...","{China, Italian}","{Diuretic, Diuretic, Expectorant, Expectorant,...","{Thuốc lợi tiểu, Thuốc long đờm, Thuốc long đờ...",Aaragus lucidus
4,Aaragus,officinalis,Liliaceae,"[China: Thuốc lợi tiểu, Thuốc diệt ký sinh trù...","{France, China, Elsewhere, Venezuela, Turkey, ...","{Cardiac, Diuretic, Diuretic, Diuretic, Diuret...","{Thuốc lợi tiểu, Thuốc diệt ký sinh trùng, Thu...",Aaragus officinalis
5,Aaragus,racemosus,Liliaceae,"[Elsewhere: Làm dịu, Thuốc lợi tiểu, Thuốc bổ,...","{Iran, Sudan, Elsewhere, India}","{Demulcent, Diuretic, Tonic, Demulcent, Demulc...","{Làm dịu, Thuốc lợi tiểu, Làm dịu, Thuốc lợi t...",Aaragus racemosus
6,Aaragus,recemosus,Liliaceae,[Elsewhere: Thuốc lợi tiểu],{Elsewhere},{Diuretic},{Thuốc lợi tiểu},Aaragus recemosus
7,Aaragus,sarmentosus,Liliaceae,[Elsewhere: Thuốc kích thích tình dục],{Elsewhere},{Aphrodisiac},{Thuốc kích thích tình dục},Aaragus sarmentosus
8,Aaragus,stipularis,Liliaceae,[Egypt: Đổ mồ hôi],{Egypt},{Diaphoretic},{Đổ mồ hôi},Aaragus stipularis
9,Abelmoschus,esculentus,Malvaceae,"[China: Làm dịu, Làm dịu, Thuốc lợi tiểu, Else...","{China, Elsewhere, Venezuela, Haiti, Iraq, Pan...","{Demulcent, Demulcent, Diuretic, Diuretic, Emo...","{Thuốc giải độc, Làm dịu, Làm dịu, Chất làm mề...",Abelmoschus esculentus


### Các hàm tạo cơ sở dữ liệu 
Cần thiết phải tải được dữ liệu về từ các nguồn cơ bản gồm
- Dữ liệu về phân loại thực vật: GIBF, WIKIDATA
- Dữ liệu thành phần hóa học: LOTUS

In [7]:
# Một số hàm cũ để phát triển ban đầu. Có thể được xóa sau khi đảm bảo kết quả thu được
import os
import requests
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import DrawingOptions
import yaml
from pygbif import species
import requests
import json
from jsonpath_ng import jsonpath, parse
import io
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import folium
from rdkit.Chem.Draw import rdMolDraw2D, rdDepictor
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG
from rdkit import rdBase
import base64
from io import BytesIO
from pygbif import species as species
from pygbif import occurrences as occ
from pygbif import maps
from mpl_toolkits.basemap import Basemap
from collections import defaultdict
import geopandas as gpd 
import folium
import matplotlib.pyplot as plt
import requests
from rdkit import Chem
from rdkit.Chem import Draw

import os
import requests
from rdkit import Chem
from rdkit.Chem import Draw
import os
import pandas as pd
from PIL import Image



# Hàm để tạo cấu trúc hóa học và lưu hình ảnh
import os
import requests
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import DrawingOptions
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
import os
import requests
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import DrawingOptions
from rdkit.Chem import PandasTools




In [ ]:
#Tải dữ liệu từ lotus database dựa trên tên loài
import os
import json
import requests
import pandas as pd

def download_compound_isolation(query, path):
    query = remove_invalid_chars(query)
    # Tạo đường dẫn tới API LOTUS
    url = f"https://lotus.naturalproducts.net/api/search/simple?query={query}"
    
    # Gửi yêu cầu và nhận kết quả dưới dạng JSON
    response = requests.get(url).json()
    
    # Chuyển đổi danh sách thành DataFrame
    compounds_df = pd.DataFrame(response['naturalProducts'])
    
    # Kiểm tra cột 'smiles' có tồn tại và không phải là None
    if 'smiles' in compounds_df.columns and compounds_df['smiles'].notnull().all():
        # Tạo thư mục nếu chưa tồn tại
        if not os.path.exists(path):
            os.makedirs(path)
        
        # Định nghĩa tên file
        filename = os.path.join(path, f"{query}.json")
        
        # Lưu dữ liệu JSON vào file
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(response, file, ensure_ascii=False, indent=4)
        
        print(f"Data for query '{query}' has been saved to '{filename}'")
    else:
        print("Column 'smiles' is missing or contains None values")
    return


if not os.path.exists(lotus_database_dir):
    os.makedirs(lotus_database_dir)

# Tạo một tập hợp (set) để lưu tên các tệp (không bao gồm đuôi)
file_names_Lotus_database = set()

# Duyệt qua các tệp trong thư mục
for name in os.listdir(lotus_database_dir):
    file_path = os.path.join(lotus_database_dir, name)
    if os.path.isfile(file_path) and name.endswith('.json'):
        # Lấy tên tệp mà không bao gồm đuôi
        file_name = os.path.splitext(name)[0]
        file_names_Lotus_database.add(file_name)

# Hiển thị kết quả


SPECIES_SET = set(df_Entha_group["Sciences_Name"])
filtered_species_set = sorted(SPECIES_SET - file_names_Lotus_database) # tạo folder dược dân tộc học
filtered_df_Entha_group_lotus = df_Entha_group.loc[df_Entha_group['Sciences_Name'].isin(filtered_species_set)]
print('Số lượng đã tải xong từ lotus:', len(file_names_Lotus_database))
print('Số lượng cần chạy tiếp:', len(filtered_species_set))
print('Số lượng loài cần chạy:', len(SPECIES_SET))
for _, row in filtered_df_Entha_group_lotus.iterrows(): 
    family, genus, species = row['FAMILY'], row['GENUS'], str(row['GENUS'] + " " + row['SPECIES'])
    download_compound_isolation(species, lotus_database_dir)

In [ ]:
### Tải dữ liệu từ GIBF
import os
import json
import pandas as pd
from pygbif import species as species
from pygbif import occurrences as occ
import requests

def get_gibf_with_images(query, path,not_found_log):
    query = remove_invalid_chars(query)
    try:
        spec = species.name_backbone(query)
    except KeyError:
        print(f"Không tìm thấy dữ liệu cho {query}")
        with open(not_found_log, 'a', encoding='utf-8') as f: 
            f.write(f"{query}\n")
        return {}, "Không có kết quả phù hợp", "Không có ghi nhận ở Việt Nam", []

    occ_global = "Không có kết quả phù hợp"
    stateProvince_in_VN = "Không có ghi nhận ở Việt Nam"
    link_image = []

    if 'usageKey' in spec:
        # Tạo thư mục nếu chưa tồn tại
        if not os.path.exists(path):
            os.makedirs(path)
        
        # Định nghĩa tên file
        filename = os.path.join(path, f"{query}.json")
        
        # Lưu dữ liệu JSON vào file
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(spec, file, ensure_ascii=False, indent=4)
        
        print(f"Dữ liệu cho truy vấn '{query}' đã được lưu vào '{filename}'")

        # Tìm kiếm các ghi nhận
        occ_search = occ.search(taxonKey=spec['usageKey'])
        occ_df = pd.DataFrame(occ_search['results'])

        # Kiểm tra sự tồn tại của cột 'country'
        if 'country' in occ_df.columns:
            # Phân bố trên thế giới
            occ_global = ", ".join(str(country) for country in set(occ_df['country']) if country)

            # Kiểm tra sự tồn tại của cột 'stateProvince'
            if 'stateProvince' in occ_df.columns:
                # Phân bố tại Việt Nam
                filtered_df = occ_df[occ_df['country'] == 'Viet Nam']
                filtered_df['stateProvince'] = filtered_df['stateProvince'].fillna('')
                stateProvince_in_VN = ", ".join(str(state) for state in set(filtered_df['stateProvince']) if state)
                if not stateProvince_in_VN:
                    stateProvince_in_VN = "Không có ghi nhận ở Việt Nam"
            else:
                print(f"Cột 'stateProvince' không tồn tại trong kết quả truy vấn cho {query}")
        else:
            print(f"Cột 'country' không tồn tại trong kết quả truy vấn cho {query}")

        # Lấy danh sách hình ảnh
        img_path = f"https://api.gbif.org/v1/occurrence/search?taxonKey={spec['usageKey']}&mediaType=StillImage&limit=5"
        response = requests.get(img_path).json()
        for result in response.get('results', []):
            for media in result.get('media', []):
                if 'identifier' in media:
                    link_image.append(media['identifier'])

        # Cập nhật nội dung JSON với thông tin mới
        json_content = {
            "spec": spec,
            "occ_global": occ_global,
            "stateProvince_in_VN": stateProvince_in_VN,
            "link_image": link_image
        }
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(json_content, file, ensure_ascii=False, indent=4)
    else:
        print(f"Không tìm thấy usageKey cho truy vấn {query}")

    return spec, occ_global, stateProvince_in_VN, link_image


not_found_log_gibf = os.path.join(GIBF_database_dir, 'not_found_gibf.csv')
# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(GIBF_database_dir):
    os.makedirs(GIBF_database_dir)

# Tạo một tập hợp (set) để lưu tên các tệp (không bao gồm đuôi)
file_names_GIBF_database = set()

# Duyệt qua các tệp trong thư mục
for name in os.listdir(GIBF_database_dir):
    file_path = os.path.join(GIBF_database_dir, name)
    if os.path.isfile(file_path) and name.endswith('.json'):
        # Lấy tên tệp mà không bao gồm đuôi
        file_name = os.path.splitext(name)[0]
        file_names_GIBF_database.add(file_name)

# Hiển thị kết quả
print('Danh sách tệp đã có trong thư mục:', file_names_GIBF_database)

# Giả sử df_Entha_group đã được khởi tạo và chứa dữ liệu
df_Entha_group['Sciences_Name'] = df_Entha_group["GENUS"] + ' ' + df_Entha_group['SPECIES']
SPECIES_SET = set(df_Entha_group["Sciences_Name"])
filtered_species_set = sorted(SPECIES_SET - file_names_GIBF_database)

filtered_df_Entha_group_gibf = df_Entha_group.loc[df_Entha_group['Sciences_Name'].isin(filtered_species_set)]
print('Số lượng đã tải xong từ GIBF:', len(file_names_GIBF_database))
print('Số lượng cần chạy tiếp:', len(filtered_species_set))
print('Số lượng loài cần chạy:', len(SPECIES_SET))

for _, row in filtered_df_Entha_group_gibf.iterrows():
    family, genus, spe = row['FAMILY'], row['GENUS'], str(row['GENUS'] + " " + row['SPECIES'])
    get_gibf_with_images(spe, GIBF_database_dir, not_found_log_gibf)


In [ ]:
# Tải dữ liệu từ WIKIDATA
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import os

def download_wikidata_for_species(species_name, path):
    species_name = remove_invalid_chars(species_name)

    """
    Truy vấn thông tin từ Wikidata về một loài thực vật dựa trên tên khoa học của loài và lưu kết quả dưới dạng tệp JSON.

    Args:
        species_name (str): Tên khoa học của loài thực vật.
        path (str): Đường dẫn thư mục để lưu tệp JSON.

    Returns:
        None
    """
    # Thiết lập endpoint SPARQL
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Truy vấn SPARQL để lấy thông tin về loài thực vật với nhãn tiếng Việt và tiếng Anh
    query = f"""
    SELECT ?item ?itemLabel ?itemLabel_vi ?itemLabel_en ?scientific_name 
           (GROUP_CONCAT(DISTINCT ?description_vi; separator=", ") AS ?descriptions_vi) 
           (GROUP_CONCAT(DISTINCT ?description_en; separator=", ") AS ?descriptions_en) 
           (GROUP_CONCAT(DISTINCT ?also_known_as_vi; separator=", ") AS ?also_known_as_vis) 
           (GROUP_CONCAT(DISTINCT ?also_known_as_en; separator=", ") AS ?also_known_as_ens) 
           ?taxonRank ?parentTaxon ?distribution ?image
    WHERE {{
      ?item wdt:P31 wd:Q16521;  # Chỉ định loài thực vật
            wdt:P225 "{species_name}".  # Tên khoa học của loài thực vật
      OPTIONAL {{ ?item wdt:P225 ?scientific_name. }}
      OPTIONAL {{ ?item schema:description ?description_vi. FILTER(LANG(?description_vi) = "vi") }}  # Mô tả bằng tiếng Việt
      OPTIONAL {{ ?item schema:description ?description_en. FILTER(LANG(?description_en) = "en") }}  # Mô tả bằng tiếng Anh
      OPTIONAL {{ ?item skos:altLabel ?also_known_as_vi. FILTER(LANG(?also_known_as_vi) = "vi") }}  # Tên khác bằng tiếng Việt
      OPTIONAL {{ ?item skos:altLabel ?also_known_as_en. FILTER(LANG(?also_known_as_en) = "en") }}  # Tên khác bằng tiếng Anh
      OPTIONAL {{ ?item wdt:P105 ?taxonRank. }}  # Hạng phân loại
      OPTIONAL {{ ?item wdt:P171 ?parentTaxon. }}  # Phân loại cha mẹ
      OPTIONAL {{ ?item wdt:P183 ?distribution. }}  # Phân bố địa lý
      OPTIONAL {{ ?item wdt:P18 ?image. }}  # Hình ảnh
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],vi,en". }}
    }}
    GROUP BY ?item ?itemLabel ?itemLabel_vi ?itemLabel_en ?scientific_name ?taxonRank ?parentTaxon ?distribution ?image
    """

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    # Chuyển kết quả truy vấn thành danh sách từ điển
    data = []
    for result in results["results"]["bindings"]:
        item = result["item"]["value"]
        itemLabel_vi = result.get("itemLabel_vi", {}).get("value", "N/A")
        itemLabel_en = result.get("itemLabel_en", {}).get("value", "N/A")
        scientific_name = result.get("scientific_name", {}).get("value", "N/A")
        descriptions_vi = result.get("descriptions_vi", {}).get("value", "N/A")
        descriptions_en = result.get("descriptions_en", {}).get("value", "N/A")
        also_known_as_vis = result.get("also_known_as_vis", {}).get("value", "N/A").split(", ")
        also_known_as_ens = result.get("also_known_as_ens", {}).get("value", "N/A").split(", ")
        taxonRank = result.get("taxonRank", {}).get("value", "N/A")
        parentTaxon = result.get("parentTaxon", {}).get("value", "N/A")
        distribution = result.get("distribution", {}).get("value", "N/A")
        image = result.get("image", {}).get("value", "N/A")
        
        # Kiểm tra và chuyển đổi URL ảnh
        if image != "N/A":
            image = image.replace("http://", "https://")  # Chuyển đổi URL thành HTTPS nếu cần

        data.append({
            "Item": item,
            "Label (VI)": itemLabel_vi,
            "Label (EN)": itemLabel_en,
            "Scientific Name": scientific_name,
            "Descriptions (VI)": descriptions_vi,
            "Descriptions (EN)": descriptions_en,
            "Also Known As (VI)": also_known_as_vis,
            "Also Known As (EN)": also_known_as_ens,
            "Taxon Rank": taxonRank,
            "Parent Taxon": parentTaxon,
            "Distribution": distribution,
            "Image": image
        })

    # Xác định đường dẫn đầy đủ để lưu tệp JSON
    output_file_path = os.path.join(path, f"{species_name.replace(' ', '_')}.json")

    # Lưu kết quả dưới dạng tệp JSON
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"Đã lưu kết quả vào {output_file_path}")



# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(WIKI_database_dir):
    os.makedirs(WIKI_database_dir)

# Tạo một tập hợp (set) để lưu tên các tệp (không bao gồm đuôi)
file_names_WIKI_database = set()

# Duyệt qua các tệp trong thư mục
for name in os.listdir(WIKI_database_dir):
    file_path = os.path.join(WIKI_database_dir, name)
    if os.path.isfile(file_path) and name.endswith('.json'):
        # Lấy tên tệp mà không bao gồm đuôi
        file_name = os.path.splitext(name)[0]
        file_names_WIKI_database.add(file_name)

# Hiển thị kết quả
print('Danh sách tệp đã có trong thư mục:', file_names_WIKI_database)

# Giả sử df_Entha_group đã được khởi tạo và chứa dữ liệu
SPECIES_SET = set(df_Entha_group["Sciences_Name"])
filtered_df_Entha_group_gibf = df_Entha_group.loc[df_Entha_group['Sciences_Name'].isin(SPECIES_SET)]
print('Số lượng đã tải xong từ WIKI:', len(file_names_WIKI_database))
print('Số lượng cần chạy tiếp:', len(SPECIES_SET - file_names_WIKI_database))
print('Số lượng loài cần chạy:', len(SPECIES_SET))

# Tải dữ liệu
for _, row in filtered_df_Entha_group_gibf.iterrows():
    family, genus, spe = row['FAMILY'], row['GENUS'], f"{row['GENUS']} {row['SPECIES']}"
    download_wikidata_for_species(spe, WIKI_database_dir)


### Xây dựng website về dược liệu

In [10]:
import os
from tabulate import tabulate


folder_names_duoc_lieu = set([name for name in os.listdir(markdocs_DL_dir) if os.path.isdir(os.path.join(markdocs_DL_dir, name))])
folder_names_duoc_dan_toc = set([name for name in os.listdir(markdocs_DTH_dir) if os.path.isdir(os.path.join(markdocs_DTH_dir, name))])
Name_file_compounds = [os.path.splitext(f)[0] for f in os.listdir(lotus_database_dir) if f.endswith('.json')]
Name_file_gibf = [os.path.splitext(f)[0] for f in os.listdir(GIBF_database_dir) if f.endswith('.json')]
Name_file_wikidata = [os.path.splitext(f)[0] for f in os.listdir(WIKI_database_dir) if f.endswith('.json')]

# Tạo DataFrame từ danh sách tên folder
TenDuocLieu_SET = set(df_CSDLHM["Tên tiếng việt dược liệu"]) # Dược liệu đã xong
filtered_TenDuocLieu_set = sorted(TenDuocLieu_SET - folder_names_duoc_lieu) # Dược liệu chưa xong

print('Số học còn lại:', len(filtered_TenDuocLieu_set))
first_two_elements = list(filtered_TenDuocLieu_set)
index_content = f""" 
!!! quote "Dược liệu là gì"
    
    - Dược liệu là nguyên liệu làm thuốc có nguồn gốc tự nhiên từ thực vật, động vật, khoáng vật và đạt tiêu chuẩn làm thuốc.

    <div style="text-align: right;"> Mục 5, điều 2, Luật dược 2016 </div>
    
    - Dược liệu (bao gồm cả vị thuốc cổ truyền) là nguyên liệu làm thuốc có nguồn gốc từ thực vật, động vật, khoáng vật, nấm và đạt tiêu chuẩn làm thuốc.
    
    <div style="text-align: right;"> Mục 5, điều 2, Luật số: 44/2024/QH15, LUẬT SỬA ĐỔI, BỔ SUNG MỘT SỐ ĐIỀU CỦA LUẬT DƯỢC </div>

<div class="grid cards" markdown>
"""
# Tạo nội dung cho update_index.yml
update_index = f"""
  - Dược liệu:
    - Dược liệu/index.md 
"""
finish_df_CSDLHM = df_CSDLHM[~df_CSDLHM["Tên tiếng việt dược liệu"].isin(first_two_elements)]

for idx, row in finish_df_CSDLHM.iterrows():
    folder_name = row["Tên tiếng việt dược liệu"]
    summary=row['Dựa tên thông tin thu thập được hãy viêt một đoạn tóm tắt ngắn về dược liệu']
    path_index = f"{project_DL}/{row['Tên tiếng việt dược liệu']}/{row['Tên tiếng việt dược liệu']}.md"
    index_content += f""" 
-   :material-leaf-circle-outline: __{row['Tên tiếng việt dược liệu']}__

    ---

    {summary}
    
    [{row['Tên tiếng việt dược liệu']}]({path_index})

"""
    
    update_index += f"    - {row['Tên tiếng việt dược liệu']}: {path_index}\n"


filtered_df_CSDLHM = df_CSDLHM[df_CSDLHM["Tên tiếng việt dược liệu"].isin(first_two_elements)]

for idx, row in filtered_df_CSDLHM.iterrows():
    folder_name = row["Tên tiếng việt dược liệu"]
    summary=row['Dựa tên thông tin thu thập được hãy viêt một đoạn tóm tắt ngắn về dược liệu']
    information_taxo=f"""
Dược liệu **{row["Tên tiếng việt dược liệu"]}** từ bộ phận **{row['Bộ_phận_dùng_Tiếng_Việt']}** từ loài *{row['Tên khoa học của Thực vật trong dược điển Việt Nam V']}*.

**Mô tả thực vật:** {row['Mô tả thực vật']}

*Tài liệu tham khảo:* {row['Mô tả thực vật bạn sử dụng tài liệu nào']} 
"""
    list_species=row["list_species"]
    string_list_species= ", ".join(list_species)
    # Đảm bảo đường dẫn tới thư mục hình ảnh và file Markdown
    species_dir = os.path.join(markdocs_DL_dir, folder_name)
    if not os.path.exists(species_dir):
        os.makedirs(species_dir)
    DDHK_path = os.path.join(species_dir, "DDHK")
    SoiBot_path = os.path.join(species_dir, "Soi_Bot")
    ViPhau_path = os.path.join(species_dir, "Vi_Phau")
    os.makedirs(DDHK_path, exist_ok=True)
    os.makedirs(SoiBot_path, exist_ok=True)
    os.makedirs(ViPhau_path, exist_ok=True)
    if len(list_species)==1:
        information_taxo+=f"Trong dược điển Việt nam, một loài được sử dụng làm dược liệu là *{row['Tên khoa học của Thực vật trong dược điển Việt Nam V']}*."
    else:
        information_taxo+=f"Trong dược điển Việt nam, một số loài có thể dùng thay thế cho nhau làm dược liệu bao gồm *{string_list_species}*"
    for Sci_name in list_species:
        if Sci_name not in Name_file_compounds:
            isolation_compound =f"Chưa có nghiên cứu về thành phần hóa học trên cơ sở Lotus"
        else:
            Sci_name = remove_invalid_chars(Sci_name)
            Name_json_file_lotus = f"{Sci_name}.json"
            path_file_json_compound_sci = os.path.join(lotus_database_dir, Name_json_file_lotus)
            if os.path.exists(path_file_json_compound_sci):
                with open(path_file_json_compound_sci, 'r', encoding='utf-8') as file:
                    data = json.load(file)   
                if 'naturalProducts' in data:
                    compounds_df = pd.DataFrame(data['naturalProducts'])
                    String_chemicalTaxonomyClassyfireClass = ""
                    String_Set_compount= ""
                    count_comp= 0
                    grouped_compound_markdown=""
                    if 'smiles' not in compounds_df.columns or compounds_df['smiles'].isnull().all():
                        isolation_compound= f"Hiện tại, chưa có thông tin về thành phần hóa học."
                    else:
                        compounds_df['Link_lotus'] = compounds_df.apply(lambda x: f"https://lotus.naturalproducts.net/compound/lotus_id/{x['lotus_id']}", axis=1)
                        compounds_df['name (code)'] = compounds_df.apply(lambda x: f"{x['traditional_name']} [({x['lotus_id']})]({x['Link_lotus']})", axis=1)
                        selected_columns_df = compounds_df.loc[:, ['lotus_id', 'traditional_name', 'synonyms','chemicalTaxonomyClassyfireClass', 'name (code)','smiles']]
                        Set_chemicalTaxonomyClassyfireClass=set(selected_columns_df['chemicalTaxonomyClassyfireClass'])
                        Set_chemicalTaxonomyClassyfireClass = {element for element in Set_chemicalTaxonomyClassyfireClass if element}
                        String_chemicalTaxonomyClassyfireClass = ', '.join(Set_chemicalTaxonomyClassyfireClass)
                        Set_compount=set(selected_columns_df['name (code)'])
                        Set_compount = {element for element in Set_compount if element}
                        String_Set_compount = ', '.join(Set_compount)
                        count_comp=len(selected_columns_df)
                        compounds_df['smiles_count'] = compounds_df['smiles'].apply(lambda x: len(x) if pd.notnull(x) else 0)
                        grouped_compounds_df = compounds_df.groupby('chemicalTaxonomyClassyfireClass').agg({ 'smiles': lambda x: list(x), 'traditional_name': list, 'lotus_id': list, 'smiles_count': 'sum', 'name (code)': list }).reset_index()
                        grouped_compounds_selected_columns = grouped_compounds_df.loc[:, ['chemicalTaxonomyClassyfireClass', 'smiles_count' ]]
                        grouped_compound_markdown=grouped_compounds_selected_columns.to_markdown(index=False) 
                        isolation_compound= f"""
**Thành phần hóa học từ loài **{Sci_name}**

Theo cơ sở dữ liệu lotus, loài *{Sci_name}* đã phân lập và xác định được **{count_comp}** hoạt chất thuộc về các nhóm {String_chemicalTaxonomyClassyfireClass} trong bảng dưới đây. Danh sách các hoạt chất như sau {String_Set_compount}. 
        
{grouped_compound_markdown}
"""
                        isolation_compound_chemical_images = []
                        for _, row1 in grouped_compounds_df.iterrows(): 
                            group = row1['chemicalTaxonomyClassyfireClass']
                            sanitized_group = sanitize_filename(group)
                            img_dir = os.path.join(markdocs_DL_dir, folder_name, Sci_name)
                            string_name_compound = ", ".join(row1['name (code)'])
                                
                            if not os.path.exists(img_dir):
                                os.makedirs(img_dir)
                                
                            smiles_list = row1['smiles']
                            chembl_mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list if smiles]
                            img_path = os.path.join(img_dir, f"{sanitized_group}.png")
                            img_path_in_mkdocs = os.path.join(Sci_name, f"{sanitized_group}.png").replace("\\", "/")
                            img = Draw.MolsToGridImage(chembl_mols, legends=row1['lotus_id'], molsPerRow=4)
                                
                            with open(img_path, 'wb') as f:
                                    f.write(img.data)
                                
                            isolation_compound_chemical_images.append(f"""
### Nhóm {group}
<figure markdown="span">
    ![{group}]({img_path_in_mkdocs}){{ width=100% }}
<figcaption>Hình ảnh cấu trúc hóa học của hoạt chất thuộc nhóm *{group}*. Tên thường gọi của các hoạt chất tương ứng là {string_name_compound}.</figcaption>
</figure>
""")

                            string_isolation_compound_chemical_images = '\n'.join(isolation_compound_chemical_images)
                            isolation_compound += f"""
            {string_isolation_compound_chemical_images}
            """
                else:
                    isolation_compound = "Hiện tại, chưa có thông tin về thành phần hóa học."
            else:
                isolation_compound = "Hiện tại, chưa có thông tin về thành phần hóa học."
                #### vẽ cấu trúc hóa học theo nhóm hoạt chất chemicalTaxonomyClassyfireClass, sau đó sẽ vẽ cấu trúc theo nhóm bằng rdkit, tạo caption cho từng nhóm

   

    # THÔNG TIN TRÊN GIBF
    
        if Sci_name not in Name_file_gibf:
            information_taxo += f""" 
Chưa có thông tin về loài này trên gibf
"""
        else:
            Name_json_file_gibf = f"{Sci_name}.json"
            path_file_json_gibf_sci = os.path.join(GIBF_database_dir, Name_json_file_gibf)
            if os.path.exists(path_file_json_gibf_sci):
                with open(path_file_json_gibf_sci, 'r', encoding='utf-8') as file:
                    data = json.load(file)        
            taxo_sci_name = data.get('spec', {}) 
            family = taxo_sci_name.get('family', 'N/A') 
            occ_global = data.get('occ_global', 'N/A')
            family = taxo_sci_name.get('family', 'N/A')
            stateProvince_in_VN = data.get('stateProvince_in_VN','N/A') 
            link_image = data.get('link_image','N/A')
            image_markdown = "\n".join([f'<img src="{link}" alt="Mô tả hình ảnh" width="100" height="100">' for link in link_image])
            information_taxo += f"""

!!! info "Phân loại thực vật của *{taxo_sci_name.get('species', 'N/A')}*"
    - **Kingdom:** {taxo_sci_name.get('kingdom', 'N/A')}
    - **Phylum:** {taxo_sci_name.get('phylum', 'N/A')}
    - **Order:** {taxo_sci_name.get('order', 'N/A')}
    - **Family:** {taxo_sci_name.get('family', 'N/A')}
    - **Genus:** {taxo_sci_name.get('genus', 'N/A')}
    - **Species:** *{taxo_sci_name.get('species', 'N/A')}*

{image_markdown}

**Phân bố trên thế giới:** {occ_global}

**Phân bố tại Việt nam:** {stateProvince_in_VN}

"""

    markdown_content = f"""---
title: {folder_name}
description: Dược liệu {folder_name} từ bộ phận {row['Bộ_phận_dùng_Tiếng_Việt']} từ loài *{row['Tên khoa học của Thực vật trong dược điển Việt Nam V']}*
date: 2024-12-01
categories:
  - {family}
tags:
  - {Sci_name}
  - {family}
---
!!! abstract "Tóm tắt"

    {row['Dựa tên thông tin thu thập được hãy viêt một đoạn tóm tắt ngắn về dược liệu']}

## Thông tin về thực vật

{information_taxo}

## Thông tin về dược liệu 

### Định danh

!!! info "Thông tin về tên gọi"

    - Dược liệu tiếng Việt: {row['Dược_liệu_Tiếng_Việt']}
    - Dược liệu tiếng Trung: {row['Dược_liệu_Tiếng_Trung_Ký_tự']} ({row['Dược_liệu_Tiếng_Trung_Phiên_âm']})
    - Dược liệu tiếng Anh: {row['Dược_liệu_Tiếng_Anh']}
    - Dược liệu latin thông dụng: {row['Dược_liệu_Tiếng_Latin']}
    - Dược liệu latin kiểu DĐVN: *{row['Dược_liệu_Tiếng_Latin_Kiểu_DĐVN']}*
    - Dược liệu latin kiểu DĐVN: *{row['Dược_liệu_Tiếng_Latin_Kiểu_DĐHK']}*
    - Dược liệu latin kiểu thông tư: *{row['Dược_liệu_Tiếng_Latin_Kiểu_Thông_Tư']}*
    - Bộ phận dùng: {row['Bộ_phận_dùng_Tiếng_Việt']} ({row['Bộ_phận_dùng_Tiếng_Anh']})

### Mô tả dược liệu 

- **Theo dược điển Việt nam V:** {row['Dược_liệu_Mô_tả_DĐVN']}

- **Mô tả dược liệu theo thông tư chế biến dược liệu theo phương pháp cổ truyền:** {row['Dược_liệu_Mô_tả_Thông_Tư']}

### Chế biến 

- **Chế biến theo dược điển việt nam V**: {row['Bộ_phận_dùng_Chế_Biến_DĐVN']}

- **Chế biến theo thông tư:** {row['Bộ_phận_dùng_Chế_Biến_Thông_Tư']}

--- 

## Thành phần hóa học

- Theo tài liệu của GS. Đỗ Tất Lợi:  {row['Thành phần hóa học']}
    
{isolation_compound}

---

## Tác dụng dược lý

Theo tài liệu {row['Mô tả thực vật bạn sử dụng tài liệu nào']}:{row['Tác dụng dược lý của Dược liệu']}

Theo tài liệu quốc tế: {row['Dược_Liệu_Tác_Dụng_Quốc_tế_Usage']}

---

## Dược điển Việt Nam V

### Soi bột:

{row['Dược_liệu_Soi_Bột_DĐVN']}

<!-- Hình ảnh soi bột sẽ được tự động chèn vào đây sau -->

### Vi phẫu:

{row['Dược_liệu_Vi_Phẫu_DĐVN']}

<!-- Hình ảnh vi phẫu sẽ được tự động chèn vào đây sau -->

### Định tính

{row['Dược_liệu_Định_Tính_DĐVN']}

### Định lượng

{row['Dược_liệu_Định_Lượng_DĐVN']}

### Thông tin khác 

- ** Độ ẩm: ** {row['Dược_liệu_Độ_ẩm_DĐVN']}
- ** Bảo quản:** {row['Dược_liệu_Bảo_Quản_DĐVN']}

## Dược điển Hồng kong

<!-- PDF sẽ được tự động chèn vào đây sau -->


---

## Y dược học cổ truyền

- **Tên vị thuốc:** {row['Tên_Vị_Thuốc']}
- **Tính vị quy kinh:** {row['Tính vị quy kinh']}
- **Công năng chủ trị:** {row['Công năng chủ trị']}
- **Chú ý:** {row['Dược_liệu_Chú_Ý_DĐVN']}
- **Kiêng kỵ:** {row['Dược_liệu_Kiêng_Kỵ_DĐVN']}

"""


    
# Lưu file content vào md file
    file_name = f"{row['Tên tiếng việt dược liệu']}.md"
    file_path = os.path.join(species_dir, file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(markdown_content)
    path_index = f"{project_DL}/{row['Tên tiếng việt dược liệu']}/{row['Tên tiếng việt dược liệu']}.md"
# thêm index_content vào file index.md
    index_content += f""" 
-   :material-leaf-circle-outline: __{row['Tên tiếng việt dược liệu']}__

    ---

    {summary}
    
    [{row['Tên tiếng việt dược liệu']}]({path_index})

"""
    
    update_index += f"    - {row['Tên tiếng việt dược liệu']}: {path_index}\n"

# Cập nhật mkdocs.yml



index_content += f"""

</div>
"""
    # Ghi nội dung vào index.md
with open(os.path.join(markdocs_DL_dir, f'index.md'), 'w', encoding='utf-8') as file:
    file.write(index_content)

    # Ghi nội dung vào update_index.yml
with open(os.path.join(markdocs_DL_dir, f'update_index.yml'), 'w', encoding='utf-8') as file:
    file.write(update_index)
     


Số học còn lại: 238


c:\Users\Admin\miniconda3\envs\wagtail\lib\site-packages\rdkit\Chem\Draw\IPythonConsole.py:261: UserWarning: Truncating the list of molecules to be displayed to 50. Change the maxMols value to display more.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\wagtail\lib\site-packages\rdkit\Chem\Draw\IPythonConsole.py:261: UserWarning: Truncating the list of molecules to be displayed to 50. Change the maxMols value to display more.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\wagtail\lib\site-packages\rdkit\Chem\Draw\IPythonConsole.py:261: UserWarning: Truncating the list of molecules to be displayed to 50. Change the maxMols value to display more.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\wagtail\lib\site-packages\rdkit\Chem\Draw\IPythonConsole.py:261: UserWarning: Truncating the list of molecules to be displayed to 50. Change the maxMols value to display more.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\wagtail\lib\site-packages\rdkit\Chem\Draw\IPythonConsole.py:261: UserWarn

# Ethanopharmacology

# Dược dân tộc học

## Version 2 cho dược dân tộc học. Dùng google trans và rút ngắn thông tin xuống
Viết lại hàm get_chemical_structure_2 thành download_compound_isolation sao cho thu được nhiều thông tin hơn
- Số lượng compound đã nghiên cứu được
- Danh sách và tìm các hoạt chất giống nhau trong cùng một họ, một chi
- Danh sách các hoạt chất khác nhau.
Như vậy sẽ cần có list danh sách các hoạt chất 
Một họ sử dụng làm thuốc sẽ cần một số thông tin sau 
- Số lượng chi sử dụng làm thuốc 
- Các bệnh sử dụng làm thuốc trong mỗi chi
- số lượng hoạt chất giống nhau trong mỗi chi. Cấu trúc hóa học. Số lượng hoạt chát trong mỗi chi.
- Phân bố trên thế giới
- Phân bố tại việt nam

# Dược dân tộc phiên bản 3
Phiên bản hai vẫn chưa tải các dữ liệu về. Cần phải viết lại cho phù hợp.
- Tải dữ liệu về lưu dưới dạng file json
- Lọc dữ liệu theo Family, Genus, Species. 
- Phân tích về cấu trúc hóa học theo họ. 
- Phân tích tác dụng dược lý.

In [8]:
df_Entha_group.head(10)

,GENUS,SPECIES,FAMILY,Country_Enth,COUNTRY,DEFINITION,ACTIVITY_VI,Sciences_Name
0,Aaragus,adscendens,Liliaceae,"[Elsewhere: Chất làm lạnh, Làm dịu, India: Thu...","{Iran, Elsewhere, India}","{Diaphoretic, Stimulant, Aphrodisiac, Tonic, R...","{Đổ mồ hôi, Chất kích thích, Chất làm lạnh, Là...",Aaragus adscendens
1,Aaragus,filicinus,Liliaceae,"[Elsewhere: Chất làm se, Thuốc bổ]",{Elsewhere},"{Astringent, Tonic}","{Chất làm se, Thuốc bổ}",Aaragus filicinus
2,Aaragus,lucidum,Liliaceae,"[China: Thuốc an thần, Thuốc bổ]",{China},"{Sedative, Tonic}","{Thuốc an thần, Thuốc bổ}",Aaragus lucidum
3,Aaragus,lucidus,Liliaceae,"[China: Thuốc lợi tiểu, Thuốc long đờm, Thuốc ...","{China, Italian}","{Diuretic, Diuretic, Expectorant, Expectorant,...","{Thuốc lợi tiểu, Thuốc long đờm, Thuốc long đờ...",Aaragus lucidus
4,Aaragus,officinalis,Liliaceae,"[China: Thuốc lợi tiểu, Thuốc diệt ký sinh trù...","{France, China, Elsewhere, Venezuela, Turkey, ...","{Cardiac, Diuretic, Diuretic, Diuretic, Diuret...","{Thuốc lợi tiểu, Thuốc diệt ký sinh trùng, Thu...",Aaragus officinalis
5,Aaragus,racemosus,Liliaceae,"[Elsewhere: Làm dịu, Thuốc lợi tiểu, Thuốc bổ,...","{Iran, Sudan, Elsewhere, India}","{Demulcent, Diuretic, Tonic, Demulcent, Demulc...","{Làm dịu, Thuốc lợi tiểu, Làm dịu, Thuốc lợi t...",Aaragus racemosus
6,Aaragus,recemosus,Liliaceae,[Elsewhere: Thuốc lợi tiểu],{Elsewhere},{Diuretic},{Thuốc lợi tiểu},Aaragus recemosus
7,Aaragus,sarmentosus,Liliaceae,[Elsewhere: Thuốc kích thích tình dục],{Elsewhere},{Aphrodisiac},{Thuốc kích thích tình dục},Aaragus sarmentosus
8,Aaragus,stipularis,Liliaceae,[Egypt: Đổ mồ hôi],{Egypt},{Diaphoretic},{Đổ mồ hôi},Aaragus stipularis
9,Abelmoschus,esculentus,Malvaceae,"[China: Làm dịu, Làm dịu, Thuốc lợi tiểu, Else...","{China, Elsewhere, Venezuela, Haiti, Iraq, Pan...","{Demulcent, Demulcent, Diuretic, Diuretic, Emo...","{Thuốc giải độc, Làm dịu, Làm dịu, Chất làm mề...",Abelmoschus esculentus


In [29]:
import pandas as pd
from rdkit.Chem import PandasTools
import os
import string
from googletrans import Translator
import networkx as nx 
import matplotlib.pyplot as plt
import requests
from rdkit.Chem import Draw
from rdkit.Chem.Draw import DrawingOptions
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import rdFMCS, Draw
from rdkit.Chem.Draw import rdMolDraw2D

import os
from tabulate import tabulate

if not os.path.exists(markdocs_DTH_dir):
    os.makedirs(markdocs_DTH_dir)


folder_names_duoc_dan_toc = set([name for name in os.listdir(markdocs_DTH_dir) if os.path.isdir(os.path.join(markdocs_DTH_dir, name))])
Name_file_compounds = [os.path.splitext(f)[0] for f in os.listdir(lotus_database_dir) if f.endswith('.json')]

Name_file_gibf = [os.path.splitext(f)[0] for f in os.listdir(GIBF_database_dir) if f.endswith('.json')]
Name_file_wikidata = [os.path.splitext(f)[0] for f in os.listdir(WIKI_database_dir) if f.endswith('.json')]

# Tạo DataFrame từ danh sách tên folder
FAMILY_SET = set(df_Entha_group["FAMILY"])
filtered_family_set = sorted(FAMILY_SET - folder_names_duoc_dan_toc)

print('Số học còn lại:', len(filtered_family_set))
first_two_elements = list(filtered_family_set)
### Nội dung mở đầu cho file index.md và update_index.yml cho các họ đã chạy xong
update_index = f"""  
  - Dược dân tộc:
    - Dược dân tộc/index.md 
"""
index_content= f"""
!!! quote "Dược dân tộc học là gì"
    
    Dược dân tộc học (Ethnopharmacology) là thuật ngữ xuất hiện năm 1967 được hiểu là môn khoa học nghiên cứu về các nguyên liệu một nhóm dân tộc sử dụng như thuốc. Nghiên cứu thuốc cổ truyền, kinh nghiệm dân gian có thể hiểu là một phần dược dân tộc nhưng nếu đặt trong ví dụ về qua trình kinh nghiệm sử dụng một loài dược liệu xuất hiện và truyền miệng trong cộng đồng dù trước đó chưa được biết tới do giao thương văn hóa làm nổi bật sự khác biệt của dược dân tộc học. Dược dân tộc học thường đề cập tới thực vật bậc cao do đó đây là nhánh của thực vật dân tộc học - chuyên nghiên cứu về ứng dụng của một loài thực vật trong đời sống hàng ngày của cộng đồng đơn giản như cỏ tranh được sử dụng để lợp mái nhà, hoặc cây sơn để làm sơn mài. Một nghiên cứu dược dân tộc học yêu cầu tập hợp kiến thức các chuyên ngành khác nhau thường bắt đầu từ điều tra kinh nghiệm sử dụng nguyên liệu làm thuốc trong không gian cộng đồng và kết thúc là nghiên cứu cung cấp bằng chứng khoa học trong phòng thí nghiệm. Dược dân tộc học đóng vai trò quan trọng chỉ dẫn đối tượng nghiên cứu trong phát triển thuốc từ thảo mộc.    
    
<div class="grid cards" markdown> 
"""
for family in folder_names_duoc_dan_toc:
    filtered_df_Entha_group_family = df_Entha_group.loc[df_Entha_group['FAMILY'] == family]
    GENUS_SET = set(filtered_df_Entha_group_family["GENUS"]) # chuyển các chi thành set
    String_genus= ', '.join(GENUS_SET) # chuyển genus set thành string, mỗi phần tử cách nhau với dấu ,
    list_country_family = set()
    for countries in filtered_df_Entha_group_family["COUNTRY"]:
        if isinstance(countries, set):
            list_country_family.update(countries)
        else:
            list_country_family.add(countries)
    list_country_family = ', '.join(list_country_family)
    #### Danh sách các bệnh được sử dụng để chữa của các loài trong họ
    list_disease_family = set()
    for disease in filtered_df_Entha_group_family["ACTIVITY_VI"]:
        if isinstance(disease, set):
            list_disease_family.update(disease)
        else:
            list_disease_family.add(disease)
    list_disease_family = sorted(set(list_disease_family))
    list_disease_family = ', '.join(list_disease_family)

    summary = f'**Họ {family}** có **{len(GENUS_SET)}** chi được các cộng đồng sử dụng trong chăm sóc sức khỏe gồm *{String_genus}*. *Họ {family}* đã được một số công động tại các quốc gia như {list_country_family} đã phát hiện một số tác dụng trên lâm sàng gồm chữa bệnh lẫn độc tính như {list_disease_family}.'
    update_index += f"""    - {family}: Dược dân tộc/{family}/{family}.md\n"""
    
    index_content += f"""

-   :material-leaf-circle-outline: __{family}__
    
    ---
    
    {summary}
    
    [Họ {family}](Dược dân tộc/{family}/{family}.md)

"""

for family in first_two_elements:
    family_dir = os.path.join(markdocs_DTH_dir, family) # tạo folder cho family
    if not os.path.exists(family_dir):
        os.makedirs(family_dir)
    filtered_df_Entha_group_family = df_Entha_group.loc[df_Entha_group['FAMILY'] == family]
    GENUS_SET = set(filtered_df_Entha_group_family["GENUS"]) # chuyển các chi thành set
    String_genus= ', '.join(GENUS_SET) # chuyển genus set thành string, mỗi phần tử cách nhau với dấu ,
    ###### Xây dựng cây phả hạ 
    ###### Xây dựng cây phả hạ 
    Tree_life = [f">Họ {family}\n"]
    count_spec_in_genus = []
    for genus in GENUS_SET:
        Tree_life.append(f"\n>|-- Chi {genus}\n")
        filtered_df_Entha_group_genus = df_Entha_group.loc[df_Entha_group['GENUS'] == genus]
        count_spec_in_genus = f'{genus} ({len(filtered_df_Entha_group_genus)}) '
        for _, row in filtered_df_Entha_group_genus.iterrows():
            Tree_life.append(f">*{row['Sciences_Name']}*,")
    Tree_life_string = '\n'.join(Tree_life)
    Background_taxonomy = f'Trong *họ {family}* có **{len(GENUS_SET)}** chi được sử dụng làm thuốc với chi tiết số loài trong mỗi chi như sau {count_spec_in_genus}. Chi tiết về loài sử dụng làm thuốc như dưới đây.  '
    #### Xử lý thông tin về dân tộc học từ cơ sở dữ liệu

    tags_spec = "\n".join([f"  - {genus} {species}" for genus, species in zip(filtered_df_Entha_group_family["GENUS"], filtered_df_Entha_group_family["SPECIES"])])
    tags_genus = "\n".join([f"  - {genus}" for genus in set(filtered_df_Entha_group_family["GENUS"])])
    #### Danh sách các quốc gia sử dụng dược liệu để chữa bệnh, dịch sang tiếng việt với công cụ google transalte
    list_country_family = set()
    for countries in filtered_df_Entha_group_family["COUNTRY"]:
        if isinstance(countries, set):
            list_country_family.update(countries)
        else:
            list_country_family.add(countries)
    list_country_family = ', '.join(list_country_family)
    #### Danh sách các bệnh được sử dụng để chữa của các loài trong họ
    list_disease_family = set()
    for disease in filtered_df_Entha_group_family["ACTIVITY_VI"]:
        if isinstance(disease, set):
            list_disease_family.update(disease)
        else:
            list_disease_family.add(disease)
    list_disease_family = sorted(set(list_disease_family))
    list_disease_family = ', '.join(list_disease_family)
    summary = f'**Họ {family}** có **{len(GENUS_SET)}** chi được các cộng đồng sử dụng trong chăm sóc sức khỏe gồm *{String_genus}*. *Họ {family}* đã được một số công động tại các quốc gia như {list_country_family} đã phát hiện một số tác dụng trên lâm sàng gồm chữa bệnh lẫn độc tính như {list_disease_family}.'
    # XỬ LÝ THÀNH PHẦN HÓA HỌC TỪ LOTUS

    # Đọc các file cấu trúc hóa học tải từ lotus dưới dạng json
    
    set_sciences_name = set(remove_invalid_chars(name) for name in filtered_df_Entha_group_family['Sciences_Name'])# Danh sách các loài trong chi
    Name_file_compounds = set([os.path.splitext(f)[0] for f in os.listdir(lotus_database_dir) if f.endswith('.json')])# Danh sách các tên loài trong chi có thể tải về từ lotus
    Information_compound = set_sciences_name.intersection(Name_file_compounds) # dược liệu có thông tin về thành phần hóa học.
    Not_informaiton_compound = set_sciences_name.difference(Name_file_compounds) # dược liệu chưa có thông tin về thành phần hóa học
    # Chỉ sử lý dược liệu có trong danh sách có thành phần hóa học
    if len(Information_compound) == 0 :
        summary_family = f'**Họ {family}** có **{len(GENUS_SET)}** chi được các cộng đồng sử dụng trong chăm sóc sức khỏe gồm *{String_genus}*. *Họ {family}* đã được một số công động tại các quốc gia như {list_country_family} đã phát hiện một số tác dụng trên lâm sàng gồm chữa bệnh lẫn độc tính như {list_disease_family}.'
        caption_image_chemical_structure = f'Chưa cấu trúc hóa học nào được phân lập tự {family}.'
        summary_isolation_compound= f'Chưa có nhiều thông tin về thành phần hóa học của họ này'
    else:
        compounds_df_final= pd.DataFrame(columns=['lotus_id', 'traditional_name', 'chemicalTaxonomyClassyfireClass','smiles', 'name (code)', 'Sci_name'])
        for SC in Information_compound: # bắt dầu duyệt tất cả các loài cho họ, lấy thành phần hóa học, đếm, sau đó tìm core 
            Sci_name = SC
            Sci_name = remove_invalid_chars(Sci_name)
            Name_json_file = f"{Sci_name}.json"
            path_file_json_compound = os.path.join(lotus_database_dir, Name_json_file)
            if os.path.exists(path_file_json_compound):
                with open(path_file_json_compound, 'r', encoding='utf-8') as file:
                    data = json.load(file)   
                compounds_df = pd.DataFrame(data['naturalProducts'])
                if 'smiles' not in compounds_df.columns or compounds_df['smiles'].isnull().all():
                    continue     
                compounds_df['name (code)'] = compounds_df.apply(lambda x: f"{x['traditional_name']} ({x['lotus_id']})", axis=1)
                compounds_df['Sci_name'] = Sci_name      
                compounds_df_final = pd.concat([compounds_df_final, compounds_df], axis=0)
        selected_columns_df = compounds_df_final.loc[:, ['lotus_id', 'traditional_name', 'chemicalTaxonomyClassyfireClass','smiles', 'name (code)', 'Sci_name']]
        Set_chemicalTaxonomyClassyfireClass=set(selected_columns_df['chemicalTaxonomyClassyfireClass'])
        Set_chemicalTaxonomyClassyfireClass = {element for element in Set_chemicalTaxonomyClassyfireClass if element}
        String_chemicalTaxonomyClassyfireClass = ', '.join(Set_chemicalTaxonomyClassyfireClass)
        grouped_selected_columns_df_sci_name = selected_columns_df.groupby('Sci_name').agg(
                smiles_list=('smiles', list),
                smiles_count=('smiles', 'count')
                ).reset_index()
        sorted_grouped_selected_columns_df_sci_name = grouped_selected_columns_df_sci_name.sort_values(by='smiles_count', ascending=False) 
        top_3_sci_name_df = sorted_grouped_selected_columns_df_sci_name.head(3) # Danh sách các dược liệu được các nhà khoa học quan tâm nhất
        top_3_sci_name_list = ', '.join(top_3_sci_name_df['Sci_name'].tolist()) # lấy tên dược liệu quan tâm chuyển sang dạng string
        def number_to_letters(n): # gán các dòng trong bảng theo thứ tự từ A-Z sao cho sau này hình ảnh in ra là theo nhóm.
            result = ''
            while n > 0:
                n -= 1
                result = chr(n % 26 + ord('A')) + result
                n //= 26
            return result   
        grouped_selected_columns_df_chemicalTaxonomyClassyfireClass = selected_columns_df.groupby('chemicalTaxonomyClassyfireClass').agg(
                        smiles_list=('smiles', list),
                        smiles_count=('smiles', 'count')
                    ).reset_index() # tính toán số lượng smiles trong mỗi nhóm
        filtered_grouped_chemicalTaxonomyClassyfireClass = grouped_selected_columns_df_chemicalTaxonomyClassyfireClass[grouped_selected_columns_df_chemicalTaxonomyClassyfireClass['smiles_count'] >= 10] # chọn các nhóm chất có số lượng hoạt chất đã được nghiên cứu lớn hơn 3 trong họ để tìm core   
        def find_core_smiles(smiles_list): # thuật toán tìm core
            mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
            mcs = rdFMCS.FindMCS(mols)
            core_smiles = Chem.MolToSmiles(Chem.MolFromSmarts(mcs.smartsString))
            return core_smiles

        filtered_grouped_chemicalTaxonomyClassyfireClass.loc[:, 'core_smiles'] = filtered_grouped_chemicalTaxonomyClassyfireClass['smiles_list'].apply(find_core_smiles)
        filtered_grouped_chemicalTaxonomyClassyfireClass['Legend'] = [number_to_letters(i + 1) for i in range(len(filtered_grouped_chemicalTaxonomyClassyfireClass))] # Tạo cột mới thể hiện thứ tự của nhóm chất. Mục tiêu đánh thứ tự từ A-Z sau này trong hình ảnh cũng như trong captopm
        smiles_list = filtered_grouped_chemicalTaxonomyClassyfireClass['core_smiles']
        chembl_mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list if smiles]
        valid_mols = [mol for mol in chembl_mols if mol is not None]
        valid_legends = [legend for mol, legend in zip(chembl_mols, filtered_grouped_chemicalTaxonomyClassyfireClass['Legend'].tolist()) if mol is not None]
        try: 
            sub_img_size = (200, 200)
            img = Draw.MolsToGridImage(valid_mols, legends=valid_legends, molsPerRow=4,subImgSize=sub_img_size,useSVG=True) # Tạo hình ảnh từ danh sách các phân tử và thêm chú thích
            caption = ', '.join(filtered_grouped_chemicalTaxonomyClassyfireClass.apply(lambda x: f"{x['chemicalTaxonomyClassyfireClass']} ({x['Legend']})", axis=1)) # tạo caption của file hình ảnh cấu trúc 
            img_path_chemical_structure = os.path.join(family_dir,'chemical_structure_core.svg') # Lưu hình ảnh
            with open(img_path_chemical_structure, 'w', encoding='utf-8') as f: 
                f.write(img.data)
        except Exception as e:
            print(f"Lỗi: {e}{family}")
                    #### Tổng kết chung về họ 
        summary_family = f'**Họ {family}** có **{len(GENUS_SET)}** chi được các cộng đồng sử dụng trong chăm sóc sức khỏe gồm *{String_genus}*. Số lượng thành phần hóa học đã phân lập và xác định cấu trúc từ họ này tính đến tháng 12 năm 2024 là **{len(set(Set_chemicalTaxonomyClassyfireClass))}** nhóm có thể liệt kê như sau *{String_chemicalTaxonomyClassyfireClass}*. Giữa các loài trong họ này, 3 dược liệu được nghiên cứu nhiều nhất dựa trên số thành phần được phân lập là **{top_3_sci_name_list}*. *Họ {family}* đã được một số công động tại các quốc gia như {list_country_family} đã phát hiện một số tác dụng trên lâm sàng gồm chữa bệnh lẫn độc tính như {list_disease_family}.'
        caption_image_chemical_structure = f' Cấu trúc hóa học của một số khung cơ bản dựa trên thuật toán FMCS để tìm {caption}.'
        summary_isolation_compound= f'Số lượng thành phần hóa học đã phân lập và xác định cấu trúc từ họ này tính đến tháng 12 năm 2024 là {len(set(Set_chemicalTaxonomyClassyfireClass))} nhóm có thể liệt kê như sau {String_chemicalTaxonomyClassyfireClass}. Số lượng các loài đã được nghiên cứu thành phần hóa học là *{len(Information_compound)}* trong tổng số *{len(Information_compound)+len(Not_informaiton_compound)}* loài thuộc họ {family}.Giữa các loài trong họ này, 3 dược liệu được nghiên cứu nhiều nhất dựa trên số thành phần được phân lập là **{top_3_sci_name_list}**. Sử dụng phần mềm RDKIT với thuật toán  Find Maximum Common Substructure (FMCS), các nhóm hoạt chất phổ biến nhất trong *họ {family}* đã xây dựng được nhân. Điều này trong tương lại có thể được sử dụng tìm kiếm mối liên hệ giữa tác dụng của cấu trúc hóa học và tác dụng dược lý. Các nhân trong phần này có thể không giống như cấu trúc gốc của từng nhóm chất. Kết quả được trình bầy như hình dưới đây.'


#### UPDATE CHO FILE index.md và update_index.md
    update_index += f"    - {family}: Dược dân tộc/{family}/{family}.md \n"
    
    index_content += f"""
-   :material-leaf-circle-outline: __{family}__
    
    ---
    
    {summary}
    
    [Họ {family}](Dược dân tộc/{family}/{family}.md)

"""


#### VIẾT MARDOWN PHẦN BAN ĐẦU

    markdown_content = f"""---
title: Các dược liệu thuộc họ {family}
description: Các dược liệu thuộc họ {family}
date: 2024-12-01
categories:
  - Dược dân tộc
  - {family}
tags:
  - Dược dân tộc
{tags_spec}
{tags_genus}
---
!!! abstract "Tóm tắt"

    {summary_family}

!!! info "DrDuke"

    James A. Duke sinh năm 1929-2017 là một nhà thực vật học người Mỹ. Đây là một trong những tác giả hàng đầu trong lĩnh vực dược dân tộc học với cuốn *CRC Handbook of Medicinal Herbs* và chính là người xây dựng lên cơ sở dữ liệu về hợp chất tự nhiên và dược dân tộc học tại Bộ nông nghiệp Hoa Kỳ. Các thông tin được đăng tải tại website [Dr. Duke's Phytochemical and Ethnobotanical Databases](https://phytochem.nal.usda.gov/). 
    Trong suốt thập niên 1970, ông lãnh đạo the Plant Taxonomy Laboratory, Plant Genetics and Germplasm Institute of the Agricultural Research Service, U.S. Department of Agriculture.
    Trong tài liệu này, các thông tin về dược dân tộc của các dược liệu được trích dẫn từ tài liệu của James A. Ducke với sự trợ giúp của phần mềm dịch thuật từ tiếng Anh sang tiếng Việt.
   
## Tổng quan về Họ {family}
### Phân loại thực vật
{Background_taxonomy}

{Tree_life_string}

### Thành phần hóa học 

{summary_isolation_compound}

<figure markdown="span">
    ![{caption_image_chemical_structure}](chemical_structure_core.svg){{ width=100% }}
    <figcaption>{caption_image_chemical_structure}</figcaption>
</figure>


!!! info  "Find Maximum Common Substructure"
    
    Thuật toán FMCS (Find Maximum Common Substructure) là một phương pháp được sử dụng để tìm ra cấu trúc chung nhiều nhất (MCS) trong một tập hợp các cấu trúc hóa học. Các bước của thuật toán gồm:
    - Chọn một cấu trúc hóa học là cấu trúc để tạo truy vấn, còn các cấu trúc khác là mục tiêu.
    - Chia nhỏ cấu trúc để tạo truy vấn thành cấu trúc nhỏ hơn dạng chuỗi SMARTS.
    - Kiểm tra chuỗi SMARTS trong các cấu trúc mục tiêu.
    - Tìm kiếm chuỗi SMARTS xuất hiện nhiều nhất.
    Để biết thêm chi tiết các bạn có thể xem tại [TeachOpenCADD](https://projects.volkamerlab.org/teachopencadd/talktorials/T006_compound_maximum_common_substructures.html)
    ``` python
    pip install rdkit
    def find_core_smiles(smiles_list):
        mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
        mcs = rdFMCS.FindMCS(mols)
        core_smiles = Chem.MolToSmiles(Chem.MolFromSmarts(mcs.smartsString))
        return core_smiles
    ```

### Dược dân tộc học

Họ **{family}** đã được một số công động tại các quốc gia như *{list_country_family}* đã phát hiện một số tác dụng trên lâm sàng gồm chữa bệnh lẫn độc tính như *{list_disease_family}*.

## Chi tiết dược dân tộc học

"""
#### VIẾT MARKDOWN DÀNH CHO CHI VÀ DƯỢC LIỆU TRONG CHI 
    for gen in GENUS_SET:
        filtered_df_Entha_group_genus = filtered_df_Entha_group_family.loc[filtered_df_Entha_group_family['GENUS'] == gen]
        set_spec_in_genus = "\n".join([f"\t - *{genus} {species}*" for genus, species in zip(filtered_df_Entha_group_genus["GENUS"], filtered_df_Entha_group_genus["SPECIES"])])
        markdown_content += f"""
### Chi {gen}

!!! note "Danh sách các loài thuộc chi"
    
*{set_spec_in_genus}*

"""
        for idx, row in filtered_df_Entha_group_genus.iterrows():
            if row['Sciences_Name'] not in Information_compound:
                isolation_compound= f"""
Chưa có nghiên cứu về thành phần hóa học của loài này
"""
            
            else:
                Sci_name = str(row['GENUS'] + " " + row['SPECIES'])
                Sci_name = remove_invalid_chars(Sci_name)
                Name_json_file_lotus = f"{Sci_name}.json"
                path_file_json_compound_sci = os.path.join(lotus_database_dir, Name_json_file_lotus)
                if os.path.exists(path_file_json_compound_sci):
                    with open(path_file_json_compound_sci, 'r', encoding='utf-8') as file:
                        data = json.load(file)   
                    compounds_df = pd.DataFrame(data['naturalProducts'])
                    String_chemicalTaxonomyClassyfireClass = ""
                    String_Set_compount= ""
                    count_comp= 0
                    grouped_compound_markdown=""
                    if 'smiles' not in compounds_df.columns or compounds_df['smiles'].isnull().all():
                        isolation_compound= f"Hiện tại, chưa có thông tin về thành phần hóa học."
                    else:
                        compounds_df['Link_lotus'] = compounds_df.apply(lambda x: f"https://lotus.naturalproducts.net/compound/lotus_id/{x['lotus_id']}", axis=1)
                        compounds_df['name (code)'] = compounds_df.apply(lambda x: f"{x['traditional_name']} [({x['lotus_id']})]({x['Link_lotus']})", axis=1)
                        selected_columns_df = compounds_df.loc[:, ['lotus_id', 'traditional_name', 'synonyms','chemicalTaxonomyClassyfireClass', 'name (code)']]
                        Set_chemicalTaxonomyClassyfireClass=set(selected_columns_df['chemicalTaxonomyClassyfireClass'])
                        Set_chemicalTaxonomyClassyfireClass = {element for element in Set_chemicalTaxonomyClassyfireClass if element}
                        String_chemicalTaxonomyClassyfireClass = ', '.join(Set_chemicalTaxonomyClassyfireClass)
                        Set_compount=set(selected_columns_df['name (code)'])
                        Set_compount = {element for element in Set_compount if element}
                        String_Set_compount = ', '.join(Set_compount)
                        count_comp=len(selected_columns_df)
                        grouped_compound_df = selected_columns_df.groupby('chemicalTaxonomyClassyfireClass').agg( lotus_count=('lotus_id', 'count') ).reset_index()
                        grouped_compound_markdown=grouped_compound_df.to_markdown(index=False) 
                        isolation_compound= f"""
Theo cơ sở dữ liệu lotus, từ loài *{Sci_name}* đã phân lập và xác định được {count_comp} hoạt chất thuộc về các nhóm {String_chemicalTaxonomyClassyfireClass}. Danh sách các hoạt chất như sau {String_Set_compount}.

{grouped_compound_markdown}
"""
            if row['Sciences_Name'] not in Name_file_gibf:
                information_taxo= f""" 
Chưa có thông tin về loài này trên gibf
"""
            else:
                Name_json_file_gibf = f"{Sci_name}.json"
                path_file_json_gibf_sci = os.path.join(GIBF_database_dir, Name_json_file_gibf)
                if os.path.exists(path_file_json_gibf_sci):
                    with open(path_file_json_gibf_sci, 'r', encoding='utf-8') as file:
                        data = json.load(file)        
                taxo_sci_name = data.get('spec', {}) 
                occ_global = data.get('occ_global', 'N/A') 
                stateProvince_in_VN = data.get('stateProvince_in_VN', 'N/A')
                link_image = data.get('link_image', 'N/A')
                image_markdown = "\n".join([f'<img src="{link}" alt="Mô tả hình ảnh" width="100" height="100">' for link in link_image])

            # Chuẩn bị dữ liệu về tên gọi và mô tả đơn giản từ wikidata
            if row['Sciences_Name'] not in Name_file_wikidata:
                infofmation_wiki = f"Chưa có thông tin về loài này trên wikidata."
            else: 
                Name_json_file_wiki = f"{Sci_name}.json"
                path_file_json_wiki_sci = os.path.join(WIKI_database_dir, Name_json_file_wiki)
                if os.path.exists(path_file_json_wiki_sci):
                    with open(path_file_json_wiki_sci, 'r', encoding='utf-8') as file:
                        data = json.load(file)
                if data: 

                    for entry in data:
                        infofmation_wiki= f"""
!!! Thông tin từ wikidata:
    - *Label (VI):* {entry.get('Label (VI)', 'N/A')}
    - *Label (EN):* {entry.get('Label (EN)', 'N/A')}
    - *Tên khoa học:* {entry.get('Scientific Name', 'N/A')}
    - *Descriptions (VI):* {entry.get('Descriptions (VI)', 'N/A')}
    - *Descriptions (EN):* {entry.get('Descriptions (EN)', 'N/A')}
    - *Also Known As (VI):* {', '.join(entry.get('Also Known As (VI)', []))}
    - *Also Known As (EN):* {', '.join(entry.get('Also Known As (EN)', []))}

"""
                        
                else:
                    infofmation_wiki = f"Chưa có thông tin về loài này trên wikidata."

            # Chuẩn bị dữ liệu Dược dân tộc học
            Coutry_enth = ';'.join(row['Country_Enth'])
            pairs = [item.strip() for item in Coutry_enth.split(';')]
            data_Coutry_enth = {'Quốc gia': [], 'Bệnh': []}
            translator = Translator()
            for pair in pairs: 
                Country, Disease = pair.split(':') 
                data_Coutry_enth['Quốc gia'].append(Country.strip()) 
                data_Coutry_enth['Bệnh'].append(Disease.strip())              
            data_Coutry_enth = pd.DataFrame(data_Coutry_enth)
            markdown_table_data_Coutry_enth = data_Coutry_enth.to_markdown(index=False)

            # Chuẩn bị dữ liệu về phân bố trong loài 


            markdown_content += f"""---      
#### *{Sci_name}*
**Thông tin về thực vật**

!!! info "Phân loại thực vật của *{taxo_sci_name.get('species', 'N/A')}* từ GIBF:"
    - **Kingdom:** {taxo_sci_name.get('kingdom', 'N/A')}
    - **Phylum:** {taxo_sci_name.get('phylum', 'N/A')}
    - **Order:** {taxo_sci_name.get('order', 'N/A')}
    - **Family:** {taxo_sci_name.get('family', 'N/A')}
    - **Genus:** {taxo_sci_name.get('genus', 'N/A')}
    - **Species:** *{taxo_sci_name.get('species', 'N/A')}*

{image_markdown} 

{infofmation_wiki}

*Phân bố trên thế giới*: {occ_global}

*Phân bố tại Việt Nam*: {stateProvince_in_VN}

**Thành phần hóa học**
        
{isolation_compound}

**Dược dân tộc học**

Danh sách các quốc gia có sử dụng *{taxo_sci_name.get('species', 'N/A')}* trong điều trị các bệnh. 

{markdown_table_data_Coutry_enth}
\n\n
"""
            
    # Đảm bảo rằng các thư mục của FAMILY tồn tại trước khi tạo file

    
    file_name = f"{family}.md"
    file_path = os.path.join(family_dir, file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(markdown_content)
    print(f"Hoàn thành tạo {file_path}")
#### Nội dung cho file index.md đóng file trước khi lưu
index_content += """
</div>
"""
# Ghi nội dung vào tệp index.md
index_file_path = os.path.join(markdocs_DTH_dir, "index.md")
with open(index_file_path, 'w', encoding='utf-8') as file:
    file.write(index_content)
print(f"Hoàn thành tạo {index_file_path}")

# Ghi nội dung vào tệp update_index.yml
update_index_file_path = os.path.join(markdocs_DTH_dir, "update_index.yml")
with open(update_index_file_path, 'w', encoding='utf-8') as file:
    file.write(update_index)
print(f"Hoàn thành tạo {update_index_file_path}")





Số học còn lại: 0
Hoàn thành tạo docs\Markdocs\Dược dân tộc\index.md
Hoàn thành tạo docs\Markdocs\Dược dân tộc\update_index.yml


In [ ]:
import os
import pandas as pd
from googletrans import Translator
# Đảm bảo thư mục 'docs' tồn tại
path = "docs"
folder = "Dược dân tộc"

if not os.path.exists(os.path.join(path, folder)):
    os.makedirs(os.path.join(path, folder))
folder_names_duoc_dan_toc = set([name for name in os.listdir(os.path.join(path, folder)) if os.path.isdir(os.path.join(path, folder, name))])
# Tạo DataFrame từ danh sách tên folder
FAMILY_SET = set(df_Entha_group["FAMILY"])
filtered_family_set = sorted(FAMILY_SET - folder_names_duoc_dan_toc)

print('Số học còn lại:', len(filtered_family_set))
first_two_elements = list(filtered_family_set)[:2]
for Family in first_two_elements:
    family_folder = os.path.join(path, folder, Family)
    if not os.path.exists(family_folder):
        os.makedirs(family_folder)
    filtered_df_Entha_group_family = df_Entha_group.loc[df_Entha_group['FAMILY'] == Family]
    tags_spec = "\n".join([f"  - {genus} {species}" for genus, species in zip(filtered_df_Entha_group_family["GENUS"], filtered_df_Entha_group_family["SPECIES"])])
    tags_genus = "\n".join([f"  - {genus}" for genus in set(filtered_df_Entha_group_family["GENUS"])])
    
    list_country_family = set()
    for countries in filtered_df_Entha_group_family["COUNTRY"]:
        if isinstance(countries, set):
            list_country_family.update(countries)
        else:
            list_country_family.add(countries)
    list_country_family = ', '.join(list_country_family)

    list_disease_family = set()
    for disease in filtered_df_Entha_group_family["DEFINITION"]:
        if isinstance(disease, set):
            list_disease_family.update(disease)
        else:
            list_disease_family.add(disease)

            
    list_disease_family = ', '.join(list_disease_family)
    translator = Translator()
    list_disease_family_vi = translator.translate(list_disease_family, dest='vi').text

##### Nội dung cho file family.md
    markdown_content = f"""---
title: Các dược liệu thuộc họ {Family}
description: Họ {Family} gồm khoảng {filtered_df_Entha_group_family["GENUS"].nunique()} chi và {filtered_df_Entha_group_family["SPECIES"].nunique()} loài được một số cộng đồng tại các quốc gia như {list_country_family} sử dụng trong hỗ trợ và điều trị bệnh {list_disease_family_vi}.
date: 2024-12-01
categories:
  - Dược dân tộc
  - {Family}
tags:
  - Dược dân tộc
{tags_spec}
{tags_genus}
---
!!! abstract "Tóm tắt"

    Họ {Family} gồm khoảng {filtered_df_Entha_group_family["GENUS"].nunique()} chi và {filtered_df_Entha_group_family["SPECIES"].nunique()} loài được một số cộng đồng tại các quốc gia như {list_country_family} sử dụng trong một số trường hợp {list_disease_family_vi}.

!!! info "DrDuke"

    James A. Duke sinh năm 1929-2017 là một nhà thực vật học người Mỹ. Đây là một trong những tác giả hàng đầu trong lĩnh vực dược dân tộc học với cuốn *CRC Handbook of Medicinal Herbs* và chính là người xây dựng lên cơ sở dữ liệu về hợp chất tự nhiên và dược dân tộc học tại Bộ nông nghiệp Hoa Kỳ. Các thông tin được đăng tải tại website [Dr. Duke's Phytochemical and Ethnobotanical Databases](https://phytochem.nal.usda.gov/). 
    Trong suốt thập niên 1970, ông lãnh đạo the Plant Taxonomy Laboratory, Plant Genetics and Germplasm Institute of the Agricultural Research Service, U.S. Department of Agriculture.
    Trong tài liệu này, các thông tin về dược dân tộc của các dược liệu được trích dẫn từ tài liệu của James A. Ducke với sự trợ giúp của phần mềm dịch thuật từ tiếng Anh sang tiếng Việt.
   
"""

    GENUS_SET = set(filtered_df_Entha_group_family["GENUS"])
    for genus in GENUS_SET:
        genus_folder = os.path.join(path, folder, Family, genus)
        if not os.path.exists(genus_folder):
            os.makedirs(genus_folder)    
        filtered_df_Entha_group_genus = filtered_df_Entha_group_family.loc[filtered_df_Entha_group_family['GENUS'] == genus]
        set_spec_in_genus = "\n".join([f"\t - *{genus} {species}*" for genus, species in zip(filtered_df_Entha_group_genus["GENUS"], filtered_df_Entha_group_genus["SPECIES"])])

        markdown_content += f"""
# Chi {genus}

??? note "Danh sách các dược liệu thuộc chi"
    
{set_spec_in_genus}

"""
        for idx, row in filtered_df_Entha_group_genus.iterrows():
            try:
                Sci_name = str(row['GENUS'] + " " + row['SPECIES'])
                Sci_name = remove_invalid_chars(Sci_name)
                species_dir = os.path.join(path, folder, Family, genus, Sci_name)
                if not os.path.exists(species_dir):
                    os.makedirs(species_dir)
                # Chuẩn bị dữ liệu Dược dân tộc học
                Coutry_enth = ';'.join(row['Country_Enth'])
                pairs = [item.strip() for item in Coutry_enth.split(';')]
                data_Coutry_enth = {'Country': [], 'Disease': [], 'Bệnh': []}
                translator = Translator()
                for pair in pairs: 
                    Country, Disease = pair.split(':') 
                    data_Coutry_enth['Country'].append(Country.strip()) 
                    data_Coutry_enth['Disease'].append(Disease.strip()) 
                    translated_disease = translator.translate(Disease.strip(), dest='vi').text
                    data_Coutry_enth['Bệnh'].append(translated_disease)                 
                data_Coutry_enth = pd.DataFrame(data_Coutry_enth)
                markdown_table_data_Coutry_enth = data_Coutry_enth.to_markdown(index=False)
                try: 
                    Sci_name_gibf = get_gibf_country(Sci_name) 
                except KeyError as e: 
                    print(f"Lỗi KeyError: {e}. Kiểm tra lại cấu trúc dữ liệu trả về từ hàm get_gibf_country.")
                try:
                    compound_img = get_chemical_structure_2(Sci_name, genus_folder)
                except KeyError as e:
                    print(f"Không thể tạo cấu trúc hóa học: {e}")
                    compound_img = ""
                if compound_img:
                    img_paths = [os.path.relpath(img, family_folder).replace("\\", "/") for img in compound_img[2]]               
                    list_compound = compound_img[1]
                    markdown_table = tabulate(list_compound[['chemicalTaxonomyClassyfireClass', 'smiles_count']], headers='keys', tablefmt='pipe')

                    def get_caption(img_name, df, col1, col2):
                        match = df[df[col1] == img_name]
                        if not match.empty:
                            return match[col2].values[0]
                        return "Không tìm thấy chú thích"
                    chemical_images_2 = "\n".join([f'''
#### Nhóm {os.path.basename(img).split('.')[0]}
<figure markdown="span">
    ![{os.path.basename(img)}]({img}){{ width=100% }}
    <figcaption>Hình ảnh cấu trúc hóa học của {get_caption(os.path.basename(img).split('.')[0], list_compound, 'chemicalTaxonomyClassyfireClass', 'smiles_count')} hoạt chất thuộc nhóm {os.path.basename(img).split('.')[0]} gồm {get_caption(os.path.basename(img).split('.')[0], list_compound, 'chemicalTaxonomyClassyfireClass', 'name (code)')}.</figcaption>
</figure>
'''.strip() for img in img_paths])

                    count_comp = len(compound_img[0])
                    list_comp = set(compound_img[0]['traditional_name'])
                    list_group_comp = ", ".join(filter(None, set(compound_img[0]['chemicalTaxonomyClassyfireClass'])))
                else:
                    count_comp = "Chưa có hoạt chất nào được phân lập."
                    chemical_images_2 = "Không có hình ảnh nào được tạo ra"
                    list_comp = "Chưa có hoạt chất nào được phân lập"
                    list_group_comp = "Không có hoạt chất nào được phân lập"
                    markdown_table = ""

                image_links = get_list_images(Sci_name)
                image_markdown = "\n".join([f'<img src="{link}" alt="Mô tả hình ảnh" width="100" height="100">' for link in image_links[:7]])
                spec = get_gibf(Sci_name)
                df_wikidata = query_wikidata_for_species(Sci_name)

                if 'Item' in df_wikidata.columns:
                    caption_table_df_wikidata = df_wikidata["Item"]
                    data_table_df_wikidata = df_wikidata.drop(columns=['Item'])
                    markdown_table_df_wikidata = data_table_df_wikidata.to_markdown(index=False)
                else:
                    print("Column 'Item' không tồn tại trong DataFrame.")
                markdown_content += f"""---
## {Sci_name}
### Thông tin về thực vật

!!! info "Phân loại thực vật của *{spec.get('species', 'N/A')}* từ GIBF:"
    - **Kingdom:** {spec.get('kingdom', 'N/A')}
    - **Phylum:** {spec.get('phylum', 'N/A')}
    - **Order:** {spec.get('order', 'N/A')}
    - **Family:** {spec.get('family', 'N/A')}
    - **Genus:** {spec.get('genus', 'N/A')}
    - **Species:** *{spec.get('species', 'N/A')}*

{image_markdown} 

![Nguồn thông tin từ wiki](caption_table_df_wikidata)

{markdown_table_df_wikidata}

#### Phân bố trên thế giới

**Từ CSDL GIBF** {Sci_name_gibf[0]}

#### Phân bố tại Việt Nam

**Từ CSDL GIBF**: {Sci_name_gibf[1]}

---
### Thành phần hóa học
        
- Theo cơ sở dữ liệu lotus: Từ loài *{spec.get('species', 'N/A')}* đã phân lập và xác định được {count_comp} hoạt chất thuộc về các nhóm {list_group_comp}. 

{markdown_table}

{chemical_images_2}

---

### Dược dân tộc học

Danh sách các quốc gia có sử dụng *{spec.get('species', 'N/A')}* trong điều trị các bệnh. 

{markdown_table_data_Coutry_enth}
\n
---

"""
            except Exception as e: 
                print(f"Lỗi tại dòng {idx}: {e}") # Tiếp tục vòng lặp với dòng tiếp theo continue    

    # Đảm bảo rằng các thư mục của FAMILY tồn tại trước khi tạo file

    
    file_name = f"{Family}.md"
    file_path = os.path.join(family_folder, file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(markdown_content)
    print(f"Hoàn thành tạo {file_path}")


### update các file md với nội dung giống nhau

In [30]:
import os

def add_giscus_to_md_files(root_folder, giscus_content):
    """
    Tìm tất cả các tệp .md trong thư mục root_folder và thêm nội dung giscus_content vào cuối mỗi tệp
    :param root_folder: Thư mục gốc để bắt đầu tìm kiếm
    :param giscus_content: Nội dung Giscus cần thêm vào các tệp .md
    """
    for folder_name, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.md'):
                file_path = os.path.join(folder_name, filename)
                with open(file_path, 'a', encoding='utf-8') as file:
                    file.write('\n' + giscus_content + '\n')
                print(f"Đã thêm nội dung vào tệp: {file_path}")

# Nội dung Giscus cần thêm vào các tệp .md
giscus_content = """
## Bình luận

<div id="giscus-container"></div>
<script src="https://giscus.app/client.js"
        data-repo="hoangson0787/CSDL-duoc-lieu"
        data-repo-id="R_kgDONbMRNA"
        data-category="Duoc lieu"
        data-category-id="DIC_kwDONbMRNM4ClklR"
        data-mapping="pathname"
        data-strict="0"
        data-reactions-enabled="1"
        data-emit-metadata="1"
        data-input-position="bottom"
        data-theme="light"
        data-lang="en"
        crossorigin="anonymous"
        async>
</script>
"""

# Thư mục gốc chứa các tệp .md
root_folder = '../docs'

# Gọi hàm để thêm nội dung Giscus vào các tệp .md
add_giscus_to_md_files(root_folder, giscus_content)


Đã thêm nội dung vào tệp: ../docs\categories.md
Đã thêm nội dung vào tệp: ../docs\index.md
Đã thêm nội dung vào tệp: ../docs\tags.md
Đã thêm nội dung vào tệp: ../docs\About\AboutMe.md
Đã thêm nội dung vào tệp: ../docs\About\AboutProject.md
Đã thêm nội dung vào tệp: ../docs\blog\index.md
Đã thêm nội dung vào tệp: ../docs\blog\posts\2024-01-12-MKDOCS.md
Đã thêm nội dung vào tệp: ../docs\blog\posts\2024-09-12-Markdown ngon ngu.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\index.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\Acanthaceae\Acanthaceae.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\Aceraceae\Aceraceae.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\Achatocarpaceae\Achatocarpaceae.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\Acoraceae\Acoraceae.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\Actinidiaceae\Actinidiaceae.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\Agavaceae\Agavaceae.md
Đã thêm nội dung vào tệp: ../docs\Dược dân tộc\Aizoaceae\Aizoaceae.md